# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [5]:
# Your code here

salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [24]:
# Your code here (I added this)

from scipy.stats import shapiro

# Filter the data for hourly workers
hourly_workers = salaries[salaries['Full or Part-Time'] == 'P']

# Perform the Shapiro-Wilk test for normality on the hourly rate
stat, p_value = shapiro(hourly_workers['Hourly Rate'].dropna())  # Adjust column name as necessary
print(f"Shapiro-Wilk Test for normality on Hourly Rate: p-value = {p_value}")

# If p-value < 0.05, data is not normally distributed
if p_value < 0.05:
    print("The data is not normally distributed. Consider using non-parametric tests.")
else:
    print("The data is normally distributed. Proceeding with t-test.")


Shapiro-Wilk Test for normality on Hourly Rate: p-value = 1.2190878727836506e-30
The data is not normally distributed. Consider using non-parametric tests.


### Challenge 2 - Normality Check

The Shapiro-Wilk test was performed to check if the hourly rate data follows a normal distribution. The p-value for the test is extremely small (**1.22e-30**), which is much lower than the significance threshold of **0.05**. 

Since the p-value is less than **0.05**, we reject the null hypothesis and conclude that the hourly rate data **is not normally distributed**. This suggests that the assumption of normality required for the t-test is violated.

As a result, we should consider using **non-parametric tests** instead of the t-test to ensure the robustness of our conclusions.


In [14]:
# Your code here

from scipy.stats import wilcoxon

# Perform the Wilcoxon signed-rank test for comparing the hourly rate to 30
# We subtract 30 from each hourly rate to check if it differs from the hypothesized value
hourly_diff = hourly_workers['Hourly Rate'].dropna() - 30

# Perform the Wilcoxon signed-rank test
stat, p_value = wilcoxon(hourly_diff)

# Output the results of the Wilcoxon signed-rank test
print(f"Wilcoxon signed-rank test statistic: {stat}")
print(f"P-value: {p_value}")



Wilcoxon signed-rank test statistic: 8692.0
P-value: 0.0


### Challenge 2 - Hypothesis Test Results

In this test, we aimed to determine whether the hourly wage of all hourly workers is significantly different from `$30` per hour. The null hypothesis is that the median hourly wage is equal to `$30`.

Since the data did not meet the normality assumption required for the t-test, we used the **Wilcoxon signed-rank test**, a non-parametric test, to compare the hourly wages to `$30`. 

The **Wilcoxon signed-rank test** is the most appropriate test in this case because:
- We are comparing the **central tendency** (median) of a single sample (hourly workers' wages) to a **fixed value** ($30), and the test accounts for non-normality in the data.
- The **Mann-Whitney U test** is not appropriate because it compares two **independent groups**, and we only have one group of hourly workers to compare to the fixed value of 30.

The results of the **Wilcoxon signed-rank test** are as follows:

- **Test statistic**: 8692.0
- **P-value**: 0.0

#### Interpretation:
- **Test statistic**: The test statistic (8692.0) represents the sum of the ranks of the absolute differences between the hourly wage and the hypothesized value of `$30`.
- **P-value**: A p-value of **0.0** is much smaller than the commonly used significance level of **0.05**. This means we **reject the null hypothesis**, indicating that the median hourly wage is significantly different from `$30/hr`.

Therefore, based on this analysis, we conclude that the hourly wage for the workers in this dataset is statistically significantly different from `$30`.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [26]:
# Your code here (I added this)

from scipy.stats import shapiro

# Perform the Shapiro-Wilk test for normality on the hourly rate
stat, p_value = shapiro(hourly_workers['Hourly Rate'].dropna())  # Adjust column name as necessary
print(f"Shapiro-Wilk Test for normality on Hourly Rate: p-value = {p_value}")

# If p-value < 0.05, data is not normally distributed
if p_value < 0.05:
    print("The data is not normally distributed. Consider using non-parametric tests.")
else:
    print("The data is normally distributed. Proceeding with t-test.")


Shapiro-Wilk Test for normality on Hourly Rate: p-value = 1.2190878727836506e-30
The data is not normally distributed. Consider using non-parametric tests.


### Challenge 3 - Normality Check

In this challenge, we performed a **Shapiro-Wilk test** to check if the hourly rate data follows a normal distribution. The p-value for the test is extremely small (**1.22e-30**), which is much lower than the significance threshold of **0.05**.

Since the p-value is less than **0.05**, we **reject the null hypothesis** and conclude that the hourly rate data **is not normally distributed**. This suggests that the assumption of normality required for methods like the **t-test** is violated.

As a result, we should consider using **non-parametric tests** (like bootstrapping or the Wilcoxon signed-rank test) to analyze the data, as these methods do not rely on the assumption of normality.


In [19]:
# Your code here

import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Define the number of bootstrap samples
n_bootstraps = 10000

# List to store the means of the bootstrap samples
bootstrap_means = []

# Generate bootstrap samples and calculate their means
for _ in range(n_bootstraps):
    sample = np.random.choice(hourly_workers['Hourly Rate'].dropna(), size=len(hourly_workers['Hourly Rate'].dropna()), replace=True)
    bootstrap_means.append(np.mean(sample))

# Calculate the 2.5% and 97.5% percentiles for the 95% confidence interval
ci_lower = np.percentile(bootstrap_means, 2.5)
ci_upper = np.percentile(bootstrap_means, 97.5)

# Output the confidence interval
print(f"95% Confidence Interval: ({ci_lower}, {ci_upper})")



95% Confidence Interval: (16.901882831181258, 17.479616212338595)


### Challenge 3 - Confidence Interval Results

In this challenge, we aimed to construct a **95% confidence interval** for the **mean hourly wage** of all hourly workers. Since the hourly wage data was not normally distributed (as confirmed by the **Shapiro-Wilk test**), we used **bootstrapping** as a non-parametric method to estimate the confidence interval for the mean.

The results of the bootstrapping are as follows:

- **95% Confidence Interval**: (16.90, 17.48)

#### Why Bootstrapping is the Most Appropriate Method:

- **Non-parametric**: Bootstrapping is a **non-parametric** method that doesn't rely on any assumptions about the distribution of the data. In this case, since the hourly wage data was found to be **non-normally distributed**, bootstrapping is an ideal choice because it doesn't require the data to follow a normal distribution.
- **Resampling with Replacement**: Bootstrapping works by repeatedly sampling with replacement from the original dataset, which allows us to approximate the sampling distribution of the mean. By generating a large number of resamples, we can calculate the confidence interval based on these simulated samples.
- **Robust to Non-normality**: Unlike methods like the t-test, which assume normality, bootstrapping remains robust even when the data is skewed, contains outliers, or violates other assumptions. This makes it especially useful when dealing with real-world data that doesn't fit well-known distributions.

#### Interpretation:
- The **95% confidence interval** means that we are 95% confident that the **true mean** of the hourly wage for all hourly workers in this dataset lies between **`$16.90`** and **`$17.48`**.
- This interval provides us with a range of plausible values for the population mean based on the sample data. Since this is a **non-parametric method**, it does not rely on the assumption of normality, making it robust even for skewed data.

Therefore, based on this confidence interval, we conclude that the hourly wage for the workers in this dataset is most likely between **`$16.90`** and **`$17.48`** per hour.


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [28]:
# Your code here (I added this)

# Number of hourly workers and total number of workers
num_hourly_workers = len(hourly_workers)
total_workers = len(salaries)

# Hypothesized proportion (25%)
hypothesized_proportion = 0.25

# Check if the sample size conditions for the z-test are met
successes = num_hourly_workers
failures = total_workers - successes

# Check the conditions
if successes >= 10 and failures >= 10:
    print("The sample size conditions are met. Proceeding with the proportions z-test.")
else:
    print("The sample size conditions are not met. Consider using an alternative method.")


The sample size conditions are met. Proceeding with the proportions z-test.


### Challenge 4 - Check Samplie Size Conditions for the Proportions Z-Test

In this challenge, we are testing whether the proportion of hourly workers in the City of Chicago is significantly different from 25%. The null hypothesis is that the proportion of hourly workers is equal to 25% (i.e., 0.25), while the alternative hypothesis is that the proportion is different.

Before performing the **proportions z-test**, we checked the **sample size conditions** to ensure that the normal approximation for the z-test is valid. Specifically, the following conditions must be met:
- **np ≥ 10** (number of successes)
- **n(1 - p) ≥ 10** (number of failures)

These conditions ensure that the sampling distribution of the proportion can be approximated by a normal distribution, making the z-test appropriate. In our case, the conditions were satisfied, and we proceeded with the **proportions z-test**.

#### Interpretation:
- The **sample size conditions** are met, meaning that the normal approximation for the z-test is valid.
- The **proportions z-test** can now be performed, and we can conclude whether the proportion of hourly workers is significantly different from 25%.

By confirming that the conditions are met, we ensure that the statistical test we apply is reliable and valid.


In [37]:
# Your code here

from statsmodels.stats.proportion import proportions_ztest

# Calculate the number of hourly workers and total number of workers
num_hourly_workers = len(hourly_workers)
total_workers = len(salaries)

# Hypothesized proportion (25%)
hypothesized_proportion = 0.25

# Perform the proportions z-test
z_stat, p_value = proportions_ztest(num_hourly_workers, total_workers, hypothesized_proportion)

# Output the results
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")


Z-statistic: -140.0706441086974
P-value: 0.0


### Challenge 4 - Hypothesis Test of Proportions Results

In this challenge, we performed a hypothesis test to determine if the proportion of hourly workers in the City of Chicago is significantly different from 25%. The null hypothesis is that the proportion of hourly workers is equal to 25%.

The results of the hypothesis test are as follows:

- **Z-statistic**: -140.07
- **P-value**: 0.0

#### Interpretation:
- **Z-statistic**: The z-statistic of **-140.07** indicates that the observed proportion of hourly workers is very far from the hypothesized proportion of 25%. A large absolute value of the z-statistic suggests a significant difference between the observed and hypothesized proportions.
- **P-value**: A p-value of **0.0** is much smaller than the commonly used significance level of **0.05**, which indicates that we **reject the null hypothesis**. This suggests that the proportion of hourly workers is significantly different from 25%.

Thus, based on the hypothesis test, we conclude that the proportion of hourly workers in the City of Chicago is significantly different from 25%.
